In [ ]:
import pennylane as qml
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import pandas as pd
import argparse
import os
import math
import datetime
import time

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch

from solver import Solver
from data_loader import get_loader
from torch.backends import cudnn
from utils import *
from models import Generator, Discriminator
from data.sparse_molecular_dataset import SparseMolecularDataset
from rdkit import Chem

def str2bool(v):
    return v.lower() in ('true')

In [2]:
parser = argparse.ArgumentParser()

# Model configuration.
parser.add_argument('--z_dim', type=int, default=8, help='dimension of domain labels')
parser.add_argument('--g_conv_dim', default=[128,256,512], help='number of conv filters in the first layer of G')
parser.add_argument('--d_conv_dim', type=int, default=[[128, 64], 128, [128, 64]], help='number of conv filters in the first layer of D')
parser.add_argument('--g_repeat_num', type=int, default=6, help='number of residual blocks in G')
parser.add_argument('--d_repeat_num', type=int, default=6, help='number of strided conv layers in D')
parser.add_argument('--lambda_cls', type=float, default=1, help='weight for domain classification loss')
parser.add_argument('--lambda_rec', type=float, default=10, help='weight for reconstruction loss')
parser.add_argument('--lambda_gp', type=float, default=10, help='weight for gradient penalty')
parser.add_argument('--post_method', type=str, default='softmax', choices=['softmax', 'soft_gumbel', 'hard_gumbel'])

# Training configuration.
parser.add_argument('--batch_size', type=int, default=128, help='mini-batch size')
parser.add_argument('--num_iters', type=int, default=10000, help='number of total iterations for training D')
parser.add_argument('--num_iters_decay', type=int, default=5000, help='number of iterations for decaying lr')
parser.add_argument('--g_lr', type=float, default=0.0001, help='learning rate for G')
parser.add_argument('--d_lr', type=float, default=0.0001, help='learning rate for D')
parser.add_argument('--dropout', type=float, default=0., help='dropout rate')
parser.add_argument('--n_critic', type=int, default=5, help='number of D updates per each G update')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for Adam optimizer')
parser.add_argument('--beta2', type=float, default=0.999, help='beta2 for Adam optimizer')
parser.add_argument('--resume_iters', type=int, default=None, help='resume training from this step')

# Test configuration.
parser.add_argument('--test_iters', type=int, default=10000, help='test model from this step')

# Miscellaneous.
parser.add_argument('--num_workers', type=int, default=1)
parser.add_argument('--mode', type=str, default='test', choices=['train', 'test'])
parser.add_argument('--use_tensorboard', type=str2bool, default=False)

# Directories.
parser.add_argument('--mol_data_dir', type=str, default='data/gdb9_9nodes.sparsedataset')
parser.add_argument('--log_dir', type=str, default='molgan/logs')
parser.add_argument('--model_save_dir', type=str, default='molgan/models')
parser.add_argument('--sample_dir', type=str, default='molgan/samples')
parser.add_argument('--result_dir', type=str, default='molgan/results')

# Step size.
parser.add_argument('--log_step', type=int, default=5)
parser.add_argument('--sample_step', type=int, default=1000)
parser.add_argument('--model_save_step', type=int, default=200)
parser.add_argument('--lr_update_step', type=int, default=1000)

config = parser.parse_known_args()[0]
print(config)

# For fast training.
cudnn.benchmark = True

Namespace(batch_size=128, beta1=0.5, beta2=0.999, d_conv_dim=[[128, 64], 128, [128, 64]], d_lr=0.0001, d_repeat_num=6, dropout=0.0, g_conv_dim=[128, 256, 512], g_lr=0.0001, g_repeat_num=6, lambda_cls=1, lambda_gp=10, lambda_rec=10, log_dir='molgan/logs', log_step=5, lr_update_step=1000, mode='test', model_save_dir='molgan/models', model_save_step=200, mol_data_dir='data/gdb9_9nodes.sparsedataset', n_critic=5, num_iters=10000, num_iters_decay=5000, num_workers=1, post_method='softmax', result_dir='molgan/results', resume_iters=None, sample_dir='molgan/samples', sample_step=1000, test_iters=10000, use_tensorboard=False, z_dim=8)


In [3]:
dev = qml.device('default.qubit', wires=8)
@qml.qnode(dev, interface='torch')
def gen_circuit(w):
    # random noise as generator input
    z = random.uniform(-1, 1)
    layers = 1
    qubits = 8
    
    # construct generator circuit for both atom vector and node matrix
    for i in range(qubits):
        qml.RY(np.arcsin(z), wires=i)
        qml.RZ(np.arccos(z), wires=i)
    for l in range(layers):
        for i in range(qubits):
            qml.RY(w[i], wires=i)
            qml.Hadamard(wires=i)
        for i in range(qubits-1):
            qml.CNOT(wires=[i, i+1])
            qml.RZ(w[i+qubits], wires=i+1)
            qml.CNOT(wires=[i, i+1])
    for i in range(qubits):
        qml.Hadamard(wires=i)
    return [qml.expval(qml.PauliZ(i)) for i in range(qubits)]

In [4]:
w = torch.tensor(list(np.random.rand(15)*2-1), requires_grad=True)
gen_circuit(w)

tensor([ 2.0857e-01,  1.1921e-01,  6.5288e-01,  4.1534e-01,  7.4812e-01,
         7.5149e-01, -6.3478e-04,  8.8258e-01], dtype=torch.float64,
       grad_fn=<_TorchQNodeBackward>)

In [5]:
self = Solver(config)

Generator(
  (layers): Sequential(
    (0): Linear(in_features=8, out_features=128, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.0, inplace=True)
    (3): Linear(in_features=128, out_features=256, bias=True)
    (4): Tanh()
    (5): Dropout(p=0.0, inplace=True)
    (6): Linear(in_features=256, out_features=512, bias=True)
    (7): Tanh()
    (8): Dropout(p=0.0, inplace=True)
  )
  (edges_layer): Linear(in_features=512, out_features=405, bias=True)
  (nodes_layer): Linear(in_features=512, out_features=45, bias=True)
  (dropoout): Dropout(p=0.0, inplace=False)
)
G
The number of parameters: 396610
Discriminator(
  (gcn_layer): GraphConvolution(
    (linear1): Linear(in_features=5, out_features=128, bias=True)
    (linear2): Linear(in_features=128, out_features=64, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (agg_layer): GraphAggregation(
    (sigmoid_linear): Sequential(
      (0): Linear(in_features=69, out_features=128, bias=True)
      (1): Sigmoid()
    )
    (tan

In [ ]:
# Learning rate cache for decaying.
g_lr = self.g_lr
d_lr = self.d_lr
gen_weights = torch.tensor([0.4368837,-0.34798056,-0.17343268,-0.63251495,-0.8964148,0.25711665,-0.54270023,0.7189626,0.8874652,0.049169622,\
                            -0.0035910252,0.7824577,-0.9482722,0.29951113,0.019537536], requires_grad=True) #16923
self.g_optimizer = torch.optim.Adam(list(self.G.parameters())+list(self.V.parameters())+[gen_weights],
                                    self.g_lr, [self.beta1, self.beta2])
self.d_optimizer = torch.optim.Adam(self.D.parameters(), self.d_lr, [self.beta1, self.beta2]) #+[gen_weights]

# Start training from scratch or resume training.
start_iters = 0
self.resume_iters = 16923
if self.resume_iters:
    start_iters = self.resume_iters
    self.restore_model(self.resume_iters)
    
# Start training.
print('Start training...')

for i in range(start_iters, 20000): #self.num_iters
    start_time = time.time()
    mols, _, _, a, x, _, _, _, _ = self.data.next_train_batch(self.batch_size)

    # =================================================================================== #
    #                             1. Preprocess input data                                #
    # =================================================================================== #

    a = torch.from_numpy(a).to(self.device).long()            # Adjacency.
    x = torch.from_numpy(x).to(self.device).long()            # Nodes.
    a_tensor = self.label2onehot(a, self.b_dim)
    x_tensor = self.label2onehot(x, self.m_dim)
    sample_list = [gen_circuit(gen_weights) for i in range(self.batch_size)]
    z = torch.stack(tuple(sample_list)).to(self.device).float()
#         z = self.sample_z(self.batch_size)
#         z = torch.from_numpy(z).to(self.device).float()

    # =================================================================================== #
    #                             2. Train the discriminator                              #
    # =================================================================================== #

    # Compute loss with real images.
    logits_real, features_real = self.D(a_tensor, None, x_tensor)
    d_loss_real = - torch.mean(logits_real)

    # Compute loss with fake images.
    edges_logits, nodes_logits = self.G(z)
    # Postprocess with Gumbel softmax
    (edges_hat, nodes_hat) = self.postprocess((edges_logits, nodes_logits), self.post_method)
    logits_fake, features_fake = self.D(edges_hat, None, nodes_hat)
    d_loss_fake = torch.mean(logits_fake)

    # Compute loss for gradient penalty.
    eps = torch.rand(logits_real.size(0),1,1,1).to(self.device)
    x_int0 = (eps * a_tensor + (1. - eps) * edges_hat).requires_grad_(True)
    x_int1 = (eps.squeeze(-1) * x_tensor + (1. - eps.squeeze(-1)) * nodes_hat).requires_grad_(True)
    grad0, grad1 = self.D(x_int0, None, x_int1)
    d_loss_gp = self.gradient_penalty(grad0, x_int0) + self.gradient_penalty(grad1, x_int1)


    # Backward and optimize.
    d_loss = d_loss_fake + d_loss_real + self.lambda_gp * d_loss_gp
    self.reset_grad()
    d_loss.backward(retain_graph=True)
    self.d_optimizer.step()

    # =================================================================================== #
    #                               3. Train the generator                                #
    # =================================================================================== #

    if (i+1) % self.n_critic == 0:
        # Z-to-target
        edges_logits, nodes_logits = self.G(z)
        # Postprocess with Gumbel softmax
        (edges_hat, nodes_hat) = self.postprocess((edges_logits, nodes_logits), self.post_method)
        logits_fake, features_fake = self.D(edges_hat, None, nodes_hat)
        g_loss_fake = - torch.mean(logits_fake)

        # Real Reward
        rewardR = torch.from_numpy(self.reward(mols)).to(self.device)
        # Fake Reward
        (edges_hard, nodes_hard) = self.postprocess((edges_logits, nodes_logits), 'hard_gumbel')
        edges_hard, nodes_hard = torch.max(edges_hard, -1)[1], torch.max(nodes_hard, -1)[1]
        mols = [self.data.matrices2mol(n_.data.cpu().numpy(), e_.data.cpu().numpy(), strict=True)
                for e_, n_ in zip(edges_hard, nodes_hard)]
        rewardF = torch.from_numpy(self.reward(mols)).to(self.device)

        # Value loss
        value_logit_real,_ = self.V(a_tensor, None, x_tensor, torch.sigmoid)
        value_logit_fake,_ = self.V(edges_hat, None, nodes_hat, torch.sigmoid)
        g_loss_value = torch.mean((value_logit_real - rewardR) ** 2 + (
                                   value_logit_fake - rewardF) ** 2)

        # Backward and optimize.
        g_loss = g_loss_fake + g_loss_value
        self.reset_grad()
        g_loss.backward(retain_graph=True)
        self.g_optimizer.step()
        print(gen_weights.detach())
        print(
            "%s\tEpoch %d/%d \t[D loss: %f]\t[G loss: %f]"
            % (datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), i+1, self.num_iters, d_loss.item(), g_loss.item())
        )

    if i == start_iters:
        g_loss = torch.tensor([-0.355871])
    try:
        n_p = np.mean([i for i in MolecularMetrics.natural_product_scores(mols, norm=True) if i != 0 ])
    except:
        n_p = 0
    try:
        qed = np.mean([i for i in MolecularMetrics.quantitative_estimation_druglikeness_scores(mols, norm=True) if i != 0 ])
    except:
        qed = 0
    try:
        logp = np.mean([i for i in MolecularMetrics.water_octanol_partition_coefficient_scores(mols, norm=True) if i != 0 ])
    except:
        logp = 0
    try:
        sa = np.mean([i for i in MolecularMetrics.synthetic_accessibility_score_scores(mols, norm=True) if i != 0 ])
    except:
        sa = 0
    try:
        valid_score = np.mean([i for i in MolecularMetrics.valid_scores(mols) if i != 0 ])
    except:
        valid_score = 0
    try:
        unique_score = np.mean([i for i in MolecularMetrics.unique_scores(mols) if i != 0 ])
    except:
        unique_core = 0        

    et = time.time() - start_time
    with open('metric_scores.csv', 'a') as file:
        writer = csv.writer(file)
        writer.writerow([i+1, et, d_loss.item(), g_loss.item()] + \
                        [n_p, qed, logp, sa, valid_score, unique_score])
    
    
    # =================================================================================== #
    #                                 4. Miscellaneous                                    #
    # =================================================================================== #
    
    # Save model checkpoints.
    if (i+1) % self.model_save_step == 0:
        G_path = os.path.join(self.model_save_dir, '{}-G.ckpt'.format(i+1))
        D_path = os.path.join(self.model_save_dir, '{}-D.ckpt'.format(i+1))
        V_path = os.path.join(self.model_save_dir, '{}-V.ckpt'.format(i+1))
        torch.save(self.G.state_dict(), G_path)
        torch.save(self.D.state_dict(), D_path)
        torch.save(self.V.state_dict(), V_path)
        with open('molgan/models/weights.csv', 'a') as file:
            writer = csv.writer(file)
            writer.writerow([i+1] + list(gen_weights.detach().numpy()))
        print('Saved model checkpoints into {}...'.format(self.model_save_dir))

    # Decay learning rates.
    if (i+1) % self.lr_update_step == 0 and (i+1) > (self.num_iters - self.num_iters_decay):
        g_lr -= (self.g_lr / float(self.num_iters_decay))
        d_lr -= (self.d_lr / float(self.num_iters_decay))
        self.update_lr(g_lr, d_lr)
        print ('Decayed learning rates, g_lr: {}, d_lr: {}.'.format(g_lr, d_lr))


Loading the trained models from step 16923...
Start training...
tensor([ 0.4368, -0.3479, -0.1733, -0.6326, -0.8965,  0.2570, -0.5428,  0.7189,
         0.8876,  0.0493, -0.0035,  0.7826, -0.9484,  0.2996,  0.0194])
2020-10-20 13:10:11	Epoch 16925/10000 	[D loss: 0.071571]	[G loss: 4.044004]


/home/junde/miniconda3/envs/kongsr-rdkit/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/junde/miniconda3/envs/kongsr-rdkit/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


tensor([ 0.4368, -0.3480, -0.1732, -0.6327, -0.8966,  0.2571, -0.5429,  0.7188,
         0.8877,  0.0493, -0.0036,  0.7826, -0.9485,  0.2997,  0.0195])
2020-10-20 13:13:16	Epoch 16930/10000 	[D loss: -0.177703]	[G loss: 4.152891]
tensor([ 0.4369, -0.3479, -0.1731, -0.6327, -0.8967,  0.2572, -0.5428,  0.7189,
         0.8878,  0.0494, -0.0037,  0.7826, -0.9486,  0.2998,  0.0196])
2020-10-20 13:16:20	Epoch 16935/10000 	[D loss: -0.143586]	[G loss: 4.190980]
tensor([ 0.4370, -0.3478, -0.1730, -0.6326, -0.8968,  0.2573, -0.5428,  0.7190,
         0.8879,  0.0494, -0.0037,  0.7825, -0.9487,  0.2999,  0.0196])
2020-10-20 13:19:25	Epoch 16940/10000 	[D loss: -0.252957]	[G loss: 4.232318]
tensor([ 0.4371, -0.3477, -0.1729, -0.6326, -0.8968,  0.2574, -0.5427,  0.7191,
         0.8880,  0.0493, -0.0038,  0.7824, -0.9488,  0.2999,  0.0196])
2020-10-20 13:22:30	Epoch 16945/10000 	[D loss: -0.129593]	[G loss: 4.334246]
tensor([ 0.4373, -0.3477, -0.1729, -0.6324, -0.8967,  0.2575, -0.5426,  0.7193,


tensor([ 0.4393, -0.3481, -0.1716, -0.6304, -0.8950,  0.2595, -0.5408,  0.7213,
         0.8901,  0.0472, -0.0051,  0.7803, -0.9506,  0.2984,  0.0187])
2020-10-20 15:04:10	Epoch 17110/10000 	[D loss: -0.048132]	[G loss: 3.755872]
tensor([ 0.4393, -0.3481, -0.1717, -0.6304, -0.8950,  0.2595, -0.5408,  0.7213,
         0.8901,  0.0472, -0.0051,  0.7803, -0.9506,  0.2983,  0.0186])
2020-10-20 15:07:24	Epoch 17115/10000 	[D loss: 0.062896]	[G loss: 3.661641]
tensor([ 0.4393, -0.3479, -0.1718, -0.6303, -0.8950,  0.2596, -0.5407,  0.7213,
         0.8901,  0.0471, -0.0051,  0.7802, -0.9507,  0.2981,  0.0184])
2020-10-20 15:10:31	Epoch 17120/10000 	[D loss: 0.011056]	[G loss: 3.786971]
tensor([ 0.4393, -0.3478, -0.1719, -0.6303, -0.8950,  0.2596, -0.5406,  0.7214,
         0.8901,  0.0470, -0.0051,  0.7802, -0.9507,  0.2979,  0.0182])
2020-10-20 15:13:35	Epoch 17125/10000 	[D loss: -0.000735]	[G loss: 3.680120]
tensor([ 0.4393, -0.3477, -0.1721, -0.6302, -0.8950,  0.2597, -0.5405,  0.7214,
  

tensor([ 0.4398, -0.3474, -0.1729, -0.6298, -0.8950,  0.2602, -0.5400,  0.7219,
         0.8897,  0.0459, -0.0061,  0.7796, -0.9507,  0.2968,  0.0169])
2020-10-20 16:51:17	Epoch 17290/10000 	[D loss: 0.239331]	[G loss: 3.116897]
tensor([ 0.4398, -0.3474, -0.1728, -0.6298, -0.8950,  0.2602, -0.5400,  0.7219,
         0.8898,  0.0459, -0.0060,  0.7796, -0.9508,  0.2968,  0.0169])
2020-10-20 16:54:12	Epoch 17295/10000 	[D loss: 0.239020]	[G loss: 3.169676]
tensor([ 0.4399, -0.3474, -0.1727, -0.6297, -0.8950,  0.2603, -0.5400,  0.7219,
         0.8899,  0.0459, -0.0061,  0.7795, -0.9508,  0.2968,  0.0170])
2020-10-20 16:57:07	Epoch 17300/10000 	[D loss: 0.238200]	[G loss: 3.241207]
tensor([ 0.4399, -0.3472, -0.1726, -0.6297, -0.8950,  0.2603, -0.5399,  0.7220,
         0.8900,  0.0460, -0.0062,  0.7795, -0.9510,  0.2968,  0.0170])
2020-10-20 17:00:01	Epoch 17305/10000 	[D loss: 0.244486]	[G loss: 3.201893]
tensor([ 0.4400, -0.3472, -0.1725, -0.6297, -0.8949,  0.2603, -0.5398,  0.7221,
    

...

tensor([ 0.4388, -0.3492, -0.1745, -0.6297, -0.8935,  0.2585, -0.5404,  0.7209,
         0.8885,  0.0458, -0.0040,  0.7800, -0.9489,  0.2957,  0.0168])
2020-10-21 01:56:07	Epoch 18190/10000 	[D loss: -0.460141]	[G loss: 2.657821]
tensor([ 0.4388, -0.3492, -0.1744, -0.6297, -0.8935,  0.2585, -0.5404,  0.7210,
         0.8886,  0.0458, -0.0040,  0.7800, -0.9489,  0.2958,  0.0169])
2020-10-21 01:59:01	Epoch 18195/10000 	[D loss: -0.456269]	[G loss: 2.700578]
tensor([ 0.4389, -0.3492, -0.1743, -0.6297, -0.8935,  0.2585, -0.5405,  0.7210,
         0.8887,  0.0459, -0.0040,  0.7800, -0.9490,  0.2958,  0.0169])
2020-10-21 02:01:54	Epoch 18200/10000 	[D loss: -0.424844]	[G loss: 2.677249]
Saved model checkpoints into molgan/models...
tensor([ 0.4389, -0.3493, -0.1742, -0.6297, -0.8935,  0.2586, -0.5405,  0.7210,
         0.8887,  0.0459, -0.0040,  0.7800, -0.9490,  0.2959,  0.0170])
2020-10-21 02:04:48	Epoch 18205/10000 	[D loss: -0.411491]	[G loss: 2.707426]
tensor([ 0.4390, -0.3493, -0.1741,

tensor([ 0.4394, -0.3489, -0.1727, -0.6301, -0.8938,  0.2589, -0.5408,  0.7212,
         0.8903,  0.0473, -0.0040,  0.7804, -0.9500,  0.2971,  0.0179])
2020-10-21 03:40:33	Epoch 18370/10000 	[D loss: 0.188098]	[G loss: 2.558045]
tensor([ 0.4393, -0.3489, -0.1728, -0.6302, -0.8939,  0.2588, -0.5409,  0.7210,
         0.8902,  0.0473, -0.0040,  0.7805, -0.9498,  0.2971,  0.0178])
2020-10-21 03:43:26	Epoch 18375/10000 	[D loss: 0.194763]	[G loss: 2.383183]
tensor([ 0.4391, -0.3489, -0.1728, -0.6303, -0.8940,  0.2586, -0.5410,  0.7209,
         0.8901,  0.0474, -0.0039,  0.7806, -0.9497,  0.2971,  0.0178])
2020-10-21 03:46:20	Epoch 18380/10000 	[D loss: 0.232603]	[G loss: 2.389222]
tensor([ 0.4389, -0.3488, -0.1728, -0.6305, -0.8941,  0.2585, -0.5411,  0.7207,
         0.8899,  0.0475, -0.0039,  0.7808, -0.9496,  0.2971,  0.0177])
2020-10-21 03:49:15	Epoch 18385/10000 	[D loss: 0.307028]	[G loss: 2.389029]
tensor([ 0.4388, -0.3487, -0.1729, -0.6307, -0.8943,  0.2584, -0.5412,  0.7206,
    

tensor([ 0.4387, -0.3492, -0.1745, -0.6296, -0.8931,  0.2585, -0.5402,  0.7210,
         0.8888,  0.0464, -0.0037,  0.7799, -0.9495,  0.2959,  0.0173])
2020-10-21 05:24:59	Epoch 18550/10000 	[D loss: -0.045712]	[G loss: 2.467265]
tensor([ 0.4386, -0.3492, -0.1745, -0.6296, -0.8931,  0.2584, -0.5403,  0.7209,
         0.8887,  0.0464, -0.0037,  0.7800, -0.9494,  0.2959,  0.0173])
2020-10-21 05:27:52	Epoch 18555/10000 	[D loss: -0.020299]	[G loss: 2.434812]
tensor([ 0.4386, -0.3491, -0.1746, -0.6297, -0.8932,  0.2584, -0.5403,  0.7209,
         0.8887,  0.0464, -0.0037,  0.7800, -0.9494,  0.2960,  0.0173])
2020-10-21 05:30:46	Epoch 18560/10000 	[D loss: 0.051148]	[G loss: 2.437004]
tensor([ 0.4385, -0.3491, -0.1746, -0.6297, -0.8932,  0.2583, -0.5404,  0.7208,
         0.8887,  0.0465, -0.0036,  0.7801, -0.9493,  0.2960,  0.0173])
2020-10-21 05:33:40	Epoch 18565/10000 	[D loss: 0.031466]	[G loss: 2.359107]
tensor([ 0.4385, -0.3491, -0.1746, -0.6297, -0.8932,  0.2582, -0.5404,  0.7207,
  

tensor([ 0.4397, -0.3495, -0.1724, -0.6295, -0.8927,  0.2590, -0.5402,  0.7215,
         0.8906,  0.0474, -0.0033,  0.7800, -0.9504,  0.2973,  0.0184])
2020-10-21 07:09:29	Epoch 18730/10000 	[D loss: -0.205241]	[G loss: 2.074363]
tensor([ 0.4398, -0.3495, -0.1723, -0.6294, -0.8926,  0.2590, -0.5402,  0.7216,
         0.8907,  0.0474, -0.0033,  0.7799, -0.9505,  0.2973,  0.0185])
2020-10-21 07:12:24	Epoch 18735/10000 	[D loss: -0.176214]	[G loss: 2.091340]
tensor([ 0.4398, -0.3495, -0.1721, -0.6294, -0.8926,  0.2591, -0.5401,  0.7217,
         0.8908,  0.0474, -0.0034,  0.7799, -0.9506,  0.2974,  0.0186])
2020-10-21 07:15:18	Epoch 18740/10000 	[D loss: -0.143935]	[G loss: 2.040758]
tensor([ 0.4399, -0.3495, -0.1720, -0.6293, -0.8925,  0.2592, -0.5401,  0.7217,
         0.8910,  0.0475, -0.0034,  0.7799, -0.9507,  0.2974,  0.0186])
2020-10-21 07:18:12	Epoch 18745/10000 	[D loss: -0.093250]	[G loss: 2.015723]
tensor([ 0.4400, -0.3495, -0.1719, -0.6293, -0.8925,  0.2592, -0.5400,  0.7218,


tensor([ 0.4399, -0.3489, -0.1718, -0.6296, -0.8929,  0.2590, -0.5402,  0.7216,
         0.8916,  0.0480, -0.0037,  0.7801, -0.9509,  0.2975,  0.0186])
2020-10-21 08:55:27	Epoch 18910/10000 	[D loss: 0.063154]	[G loss: 1.506055]
tensor([ 0.4398, -0.3489, -0.1720, -0.6297, -0.8929,  0.2589, -0.5402,  0.7215,
         0.8914,  0.0480, -0.0037,  0.7802, -0.9508,  0.2974,  0.0185])
2020-10-21 08:58:42	Epoch 18915/10000 	[D loss: 0.034484]	[G loss: 1.483382]
tensor([ 0.4396, -0.3488, -0.1723, -0.6297, -0.8930,  0.2588, -0.5403,  0.7214,
         0.8912,  0.0480, -0.0037,  0.7802, -0.9506,  0.2973,  0.0185])
2020-10-21 09:01:49	Epoch 18920/10000 	[D loss: 0.112075]	[G loss: 1.562231]
tensor([ 0.4395, -0.3488, -0.1724, -0.6298, -0.8931,  0.2587, -0.5403,  0.7213,
         0.8910,  0.0479, -0.0036,  0.7803, -0.9504,  0.2972,  0.0184])
2020-10-21 09:04:56	Epoch 18925/10000 	[D loss: 0.196958]	[G loss: 1.583534]
tensor([ 0.4393, -0.3488, -0.1726, -0.6299, -0.8932,  0.2586, -0.5404,  0.7212,
    

In [ ]:
i+1

In [ ]:
# For saving models
G_path = os.path.join(self.model_save_dir, '{}-G.ckpt'.format(16923))
D_path = os.path.join(self.model_save_dir, '{}-D.ckpt'.format(16923))
V_path = os.path.join(self.model_save_dir, '{}-V.ckpt'.format(16923))
torch.save(self.G.state_dict(), G_path)
torch.save(self.D.state_dict(), D_path)
torch.save(self.V.state_dict(), V_path)
with open('molgan/models/weights.csv', 'a') as file:
    writer = csv.writer(file)
    writer.writerow([i+1] + list(gen_weights.detach().numpy()))
print('Saved model checkpoints into {}...'.format(self.model_save_dir))
gen_weights # 4951 [ 0.4384, -0.3454, -0.1851, -0.6287, -0.8948,  0.2568, -0.5385,  0.7215, 0.8844,  0.0439, -0.0106,  0.7786, -0.9480,  0.2930,  0.0160]

In [ ]:
sample_list = [gen_circuit(gen_weights) for i in range(self.batch_size)]
z = torch.stack(tuple(sample_list)).to(self.device).float()
edges_logits, nodes_logits = self.G(z)

(edges_hard, nodes_hard) = self.postprocess((edges_logits, nodes_logits), 'hard_gumbel')
edges_hard, nodes_hard = torch.max(edges_hard, -1)[1], torch.max(nodes_hard, -1)[1]
mols = [self.data.matrices2mol(n_.data.cpu().numpy(), e_.data.cpu().numpy(), strict=True)
        for e_, n_ in zip(edges_hard, nodes_hard)]

In [ ]:
[i for i in range(len(mols)) if mols[i] != None]

In [ ]:
nodes_hard[61]

In [ ]:
edges_hard[61]

In [ ]:
from rdkit.Chem.Draw import SimilarityMaps
import matplotlib

for j in range(50, 62):
    mol = mols[j]
    AllChem.ComputeGasteigerCharges(mol)
    contribs = [mol.GetAtomWithIdx(i).GetDoubleProp('_GasteigerCharge') for i in range(mol.GetNumAtoms())]
    fig = SimilarityMaps.GetSimilarityMapFromWeights(mol, contribs, colorMap=None,  contourLines=10)

In [ ]:
def reward(self, mols):
    rr = 1.
    for m in ('logp,sas,qed,unique' if self.metric == 'all' else self.metric).split(','):

        if m == 'np':
            rr *= MolecularMetrics.natural_product_scores(mols, norm=True)
        elif m == 'logp':
            rr *= MolecularMetrics.water_octanol_partition_coefficient_scores(mols, norm=True)
        elif m == 'sas':
            rr *= MolecularMetrics.synthetic_accessibility_score_scores(mols, norm=True)
        elif m == 'qed':
            rr *= MolecularMetrics.quantitative_estimation_druglikeness_scores(mols, norm=True)
        elif m == 'novelty':
            rr *= MolecularMetrics.novel_scores(mols, data)
        elif m == 'dc':
            rr *= MolecularMetrics.drugcandidate_scores(mols, data)
        elif m == 'unique':
            rr *= MolecularMetrics.unique_scores(mols)
        elif m == 'diversity':
            rr *= MolecularMetrics.diversity_scores(mols, data)
        elif m == 'validity':
            rr *= MolecularMetrics.valid_scores(mols)
        else:
            raise RuntimeError('{} is not defined as a metric'.format(m))

    return rr.reshape(-1, 1)

In [ ]:
np.mean([i for i in MolecularMetrics.natural_product_scores(mols, norm=True) if i != 0 ])

In [ ]:
np